In [1]:
import os
import splusdata

In [2]:
os.system('./galfitm-1.4.4-osx galfit.feedme.template')

26880

In [3]:
DATA_FOLDER = 'data'
OUTPUT_FOLDER = 'output'

In [4]:
conn = splusdata.connect()

You have access to internal data


In [29]:
name = 'test'
ra = 41.3649
dec = -0.9144

In [51]:
for band in ['R', 'I', 'G']:
    #conn.get_cut(ra, dec, 200, band, filepath=os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'))
    #unpack_file(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits.fz'))

    make_psf(os.path.join(DATA_FOLDER, f'{name}_{band.lower()}.fits'), outfile=os.path.join(DATA_FOLDER, f'psf_{name}_{band.lower()}.fits'))

In [7]:
df = conn.query(f"""

SELECT r.id, r.ra, r.dec, r.r_petro, g.g_petro, i.i_petro 

from "idr4_dual"."idr4_dual_r" as r JOIN 
"idr4_dual"."idr4_dual_g" as g ON g.ID = r.ID JOIN
"idr4_dual"."idr4_dual_i" as i ON i.ID = r.ID 

where 1 = CONTAINS( POINT('ICRS', r.ra, r.dec), 
CIRCLE('ICRS', {ra}, {dec}, 0.003))

""")

finished


In [40]:
import astropy
from astropy.io import fits

def unpack_file(filename, output=None, output_folder=None, delete_compressed = False):
    import os
    """
    Funpack (uncompress) .fz file
    """

    if output_folder:
        output = os.path.join(output_folder, filename.replace('.fz', ''))
    elif not output:
        output = filename.replace('.fz', '')
        
    if delete_compressed:
        os.remove(filename)
    
    packed = fits.open(filename)
    unpacked = astropy.io.fits.hdu.image.PrimaryHDU(data = packed[1].data, header = packed[1].header)
    astropy.io.fits.hdu.hdulist.HDUList(hdus=[unpacked]).writeto(output, overwrite=True)

In [47]:
import numpy as np
from astropy.io import fits as pf

def make_psf(filename, outfile=None, fwhm=None, beta=None, radius = 10):
    
    if not fwhm or beta:
        psf_data = get_psf_data(filename)
        fwhm = psf_data[0]
        beta = psf_data[1]

    fwhm = fwhm/0.5
    #beta = beta/0.5 
    alpha = fwhm / (2 * np.sqrt(np.power(2., 1/beta) - 1.))
    r = np.linspace(-radius, radius, 2 * radius + 1)
    X, Y = np.meshgrid(r, r)
    R = np.sqrt(X**2 + Y**2)
    I = (beta - 1.) / (np.pi * alpha**2) * \
        np.power(1. + np.power(R / alpha, 2), -beta)
    
    if outfile:
        hdu = fits.PrimaryHDU(I)
        hdulist = pf.HDUList([hdu])
        hdulist.writeto(outfile, overwrite=True)

    return I

In [12]:
def get_psf_data(filename):
    """Returns x0,y0 from image header"""
    hdulist = fits.open(filename)
    hdu = hdulist[0]
    
    ret = [None, None]
    for i in hdu.header:
        if 'FWHMMEAN' in i:
            ret[0] = hdu.header[i]
        if 'FWHMBETA' in i:
            ret[1] = hdu.header[i]
            
    hdulist.close()
    return tuple(ret)

In [13]:
def solve_feedme_template(template, search_replace):
    """
    Replaces all patterns in template text, returning a (hopefully) sane GALFIT FEEDME text

    Args:
        template: template text
        search_replace: [(search0, replace0), (search1, replace1), ...], i.e., search-replace pairs

    Returns:
        str: contents
    """

    contents = template
    for pattern, str_replace in search_replace:
        contents = contents.replace(pattern, str(str_replace))

    return contents

In [14]:
def get_dims(filename):
    """Returns a tuple (width, height)"""
    hdulist = fits.open(filename)
    hdu = hdulist[0]
    ret = (hdu.header["NAXIS1"], hdu.header["NAXIS2"])
    hdulist.close()
    return ret

def get_exptime(filename):
    """Returns exposure-time from image header"""
    hdulist = fits.open(filename)
    hdu = hdulist[0]
    ret = hdu.header["EXPTIME"]
    hdulist.close()
    return ret

In [16]:
width, height = get_dims('G.fits')

In [17]:
import numpy as np

zpg = 25.11 - 2.5 * np.log10(
    float(get_exptime("G.fits")))
zpr = 24.80 - 2.5 * np.log10(
    float(get_exptime("R.fits")))
zpi = 24.36 - 2.5 * np.log10(
    float(get_exptime("I.fits")))


x0g, y0g = get_dims("G.fits")
x0r, y0r = get_dims("R.fits")
x0i, y0i = get_dims("I.fits")

In [52]:
search_replace = [
            ("@@@@@@", os.path.join(DATA_FOLDER, name)),
            ("WWWWWW", str(width)),
            ("HHHHHH", str(height)),
            ("ZPG", str(float(zpg))),
            ("ZPR", str(float(zpr))),
            ("ZPI", str(float(zpi))),
            ("XGXGXG", str(float(x0g))),
            ("XRXRXR", str(float(x0r))),
            ("XIXIXI", str(float(x0i))),
            ("YGYGYG", str(float(y0g))),
            ("YRYRYR", str(float(y0r))),
            ("YIYIYI", str(float(y0i))),
            #("BKGG", str(float(row["skybg_g"])/nmgyg)),
            #("BKGR", str(float(row["skybg_r"])/nmgyr)),
            #("BKGI", str(float(row["skybg_i"])/nmgyi)),
            #("BKGU", str(float(skybg_u))),
            #("BKGG", str(float(skybg_g))),
            #("BKGR", str(float(skybg_r))),
            #("BKGI", str(float(skybg_i))),
            #("BKGZ", str(float(skybg_z))),
            ("MMAGG", df[0]['g_petro']),
            ("MMAGR", df[0]['r_petro']),
            ("MMAGI", df[0]['i_petro']),
        ]

In [53]:
template = open('galfit.feedme.template', 'r').read()

In [54]:
n_template = solve_feedme_template(template, search_replace)

In [55]:
f = open('galfit.feedme', 'w')
f.write(n_template)
f.close()

In [56]:
os.system('./galfitm-1.4.4-osx galfit.feedme')

26880